# Lab 9: Tracing and Observability for AI Agents

Build **production-ready observability** for your AI agents using OpenTelemetry and Azure Monitor.

## Learning Objectives

By the end of this lab, you will:
1. Understand **distributed tracing concepts** and why they matter for AI agents
2. Configure **Azure Monitor** for production trace collection
3. Instrument **OpenAI SDK calls** with automatic tracing
4. Create **custom spans** for business logic
5. Analyze traces in the **Foundry portal**

## Prerequisites

| Requirement | Setup |
|------------|-------|
| Labs 1-8 completed | Agent patterns, evaluations |
| Python environment | Run `uv sync --all-extras --dev` from project root |
| Azure CLI logged in | Run `az login` in terminal |
| Azure AI Foundry Project | [Create at ai.azure.com](https://ai.azure.com) |
| Application Insights | Connected to your Foundry project |

## Conceptual Foundation: Distributed Tracing

### What is a Trace?

A **trace** represents the complete execution path of a single request through your system. In the context of AI agents:

```
User Query: "What's the weather in Seattle and calculate a 20% tip on $85"
                                    │
                                    ▼
┌─────────────────────────────────────────────────────────────────────────┐
│                         TRACE: request_abc123                           │
│                                                                         │
│  ┌─────────────────────────────────────────────────────────────────┐   │
│  │ SPAN: agent_session (root span)                     2.3s total  │   │
│  │                                                                  │   │
│  │  ┌────────────────────────────────────────────────────────────┐ │   │
│  │  │ SPAN: planning                                    800ms    │ │   │
│  │  │   └── SPAN: chat.completions (auto-instrumented)  750ms    │ │   │
│  │  │         attributes: model=gpt-4o, tokens.input=45          │ │   │
│  │  └────────────────────────────────────────────────────────────┘ │   │
│  │                                                                  │   │
│  │  ┌────────────────────────────────────────────────────────────┐ │   │
│  │  │ SPAN: tool_execution                              100ms    │ │   │
│  │  │   ├── SPAN: tool_fetch_weather                     40ms    │ │   │
│  │  │   │     attributes: tool.args={location: Seattle}          │ │   │
│  │  │   └── SPAN: tool_calculate_tip                     50ms    │ │   │
│  │  │         attributes: tool.args={bill: 85, pct: 20}          │ │   │
│  │  └────────────────────────────────────────────────────────────┘ │   │
│  │                                                                  │   │
│  │  ┌────────────────────────────────────────────────────────────┐ │   │
│  │  │ SPAN: response_generation                         1.2s     │ │   │
│  │  │   └── SPAN: chat.completions (auto-instrumented)  1.1s     │ │   │
│  │  │         attributes: model=gpt-4o, tokens.output=120        │ │   │
│  │  └────────────────────────────────────────────────────────────┘ │   │
│  │                                                                  │   │
│  └─────────────────────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────────────────────┘
```

### Key Terminology

| Term | Definition | AI Agent Example |
|------|------------|------------------|
| **Trace** | A directed acyclic graph (DAG) of spans representing one request | Complete user query → response flow |
| **Span** | A single unit of work with start time, duration, and metadata | One LLM call, one tool execution |
| **Parent-Child** | Spans have hierarchical relationships | `agent_session` → `planning` → `chat.completions` |
| **Attributes** | Key-value metadata attached to spans | `model.name="gpt-4o"`, `tokens.input=45` |
| **Context Propagation** | Passing trace IDs across service boundaries | Ensures all spans link to same trace |

### Why Traces Matter for AI Agents

| Problem | Without Tracing | With Tracing |
|---------|-----------------|--------------|
| **Debugging** | "The agent gave wrong answer" | See exact prompt, model response, tool outputs at each step |
| **Performance** | "It's slow sometimes" | Identify bottleneck: was it the LLM (2s) or tool call (50ms)? |
| **Cost Analysis** | "How much did that cost?" | Sum `tokens.input` + `tokens.output` across all LLM spans |
| **Error Diagnosis** | "Something failed" | Pinpoint exact span with exception, see preceding context |
| **Behavior Audit** | "Why did it do that?" | Trace shows: LLM decided to call tools X, Y; got results; synthesized |

## OpenTelemetry: The Standard

**OpenTelemetry (OTel)** is the CNCF standard for observability. It provides:

1. **Unified API**: Same code works with any backend (Azure Monitor, Jaeger, etc.)
2. **Auto-Instrumentation**: Libraries like `opentelemetry-instrumentation-openai-v2` automatically create spans
3. **Semantic Conventions**: Standardized attribute names (e.g., `gen_ai.request.model`)

### OpenTelemetry Architecture

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         Your Application                                    │
│                                                                             │
│   ┌─────────────────┐    ┌─────────────────┐    ┌─────────────────┐        │
│   │   Your Code     │    │  OpenAI SDK     │    │ Azure AI SDK    │        │
│   │                 │    │                 │    │                 │        │
│   │ tracer.start_   │    │ (auto-instru-   │    │ (auto-instru-   │        │
│   │ as_current_span │    │  mented)        │    │  mented)        │        │
│   └────────┬────────┘    └────────┬────────┘    └────────┬────────┘        │
│            │                      │                      │                  │
│            └──────────────────────┼──────────────────────┘                  │
│                                   ▼                                         │
│                    ┌──────────────────────────────┐                         │
│                    │      TracerProvider          │                         │
│                    │  • Creates spans             │                         │
│                    │  • Manages context           │                         │
│                    └──────────────┬───────────────┘                         │
│                                   │                                         │
│                    ┌──────────────▼───────────────┐                         │
│                    │      SpanProcessor           │                         │
│                    │  • BatchSpanProcessor        │                         │
│                    │  • SimpleSpanProcessor       │                         │
│                    └──────────────┬───────────────┘                         │
│                                   │                                         │
│                    ┌──────────────▼───────────────┐                         │
│                    │        Exporter              │                         │
│                    │  • AzureMonitorExporter      │                         │
│                    │  • OTLPSpanExporter          │                         │
│                    │  • ConsoleSpanExporter       │                         │
│                    └──────────────┬───────────────┘                         │
└───────────────────────────────────┼─────────────────────────────────────────┘
                                    │
                    ┌───────────────┴───────────────┐
                    ▼                               ▼
        ┌───────────────────┐           ┌───────────────────┐
        │   Azure Monitor   │           │   OTLP Endpoint   │
        │ (App Insights)    │           │ (AI Toolkit, etc) │
        └───────────────────┘           └───────────────────┘
```

### Key Insight: One TracerProvider Per Process

⚠️ **Important**: OpenTelemetry allows only **one `TracerProvider`** per Python process. This means:
- You must choose Azure Monitor OR AI Toolkit (not both simultaneously)
- To switch backends, restart your kernel/process
- This is why we have separate notebooks for each backend

## Environment Variables

```bash
# Required in .env file
AZURE_AI_PROJECT_ENDPOINT=https://your-project.services.ai.azure.com/api/projects/your-project
AZURE_AI_MODEL_DEPLOYMENT_NAME=gpt-4o
```

## Choosing a Tracing Backend

This lab uses **Azure Monitor** (production-grade). For local-only development, see the separate notebook `lab9b_aitk_tracing.ipynb`.

### Azure Monitor vs AI Toolkit: When to Use Each

| Aspect | Azure Monitor (This Lab) | AI Toolkit (lab9b_aitk_tracing.ipynb) |
|--------|--------------------------|--------------------------------------|
| **Use Case** | Production, team collaboration | Local development, quick iteration |
| **Setup** | Requires Azure subscription + App Insights | Just VS Code extension |
| **Data Retention** | 90 days (configurable) | Session only (not persisted) |
| **Query/Analysis** | KQL queries, dashboards, alerts | Visual inspection only |
| **Cost** | Pay per GB ingested | Free |
| **Latency** | 1-2 min to appear in portal | Instant in VS Code |
| **Best For** | Debugging production issues, auditing | Rapid prototyping, learning |

### Technical Difference

Both use OpenTelemetry, but with different **Exporters**:

```python
# Azure Monitor: Uses Azure-specific exporter
from azure.monitor.opentelemetry import configure_azure_monitor
configure_azure_monitor(connection_string="InstrumentationKey=...")

# AI Toolkit: Uses generic OTLP HTTP exporter
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
exporter = OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
```

> 📚 **Documentation**: 
> - [Trace and observe AI agents](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-observe-agents)
> - [AI Toolkit Tracing](https://code.visualstudio.com/docs/intelligentapps/tracing)

# Part 1: Setup and Environment

In [1]:
import os
import json
from pathlib import Path
from dotenv import load_dotenv

# Ensure Azure CLI is in PATH
for p in ["/opt/homebrew/bin", "/usr/local/bin"]:
    if p not in os.environ.get("PATH", ""):
        os.environ["PATH"] = p + ":" + os.environ.get("PATH", "")

# Load .env
env_path = Path("../.env")
if env_path.exists():
    load_dotenv(env_path)
    print(f"✅ Loaded {env_path.resolve()}")

endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")
model = os.getenv("AZURE_AI_MODEL_DEPLOYMENT_NAME", "gpt-4o")

print(f"✅ Endpoint: {endpoint[:50]}..." if endpoint else "❌ Missing AZURE_AI_PROJECT_ENDPOINT")
print(f"✅ Model: {model}")

✅ Loaded /Users/pablo/Desktop/githubRepos/teaching/northwestern/northwestern-fy26-msai-foundry-agentic-ai/.env
✅ Endpoint: https://nw-fy-26.services.ai.azure.com/api/project...
✅ Model: gpt-4.1


In [2]:
# Install tracing packages
# Run in terminal: uv pip install azure-ai-projects azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2

# Core imports
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

# OpenTelemetry imports
from opentelemetry import trace

print("✅ All imports successful")

✅ All imports successful


# Part 2: Enable Content Capture (Optional)

By default, OpenTelemetry instrumentation does **not** capture the actual content of messages (for privacy). To see full inputs/outputs, enable content capture:

> ⚠️ **Security Warning**: Tracing can capture sensitive information (user inputs, model outputs, tool arguments). Use these practices to reduce risk:
> - Don't store secrets, credentials, or tokens in prompts, tool arguments, or span attributes
> - Redact or minimize personal data before it appears in telemetry
> - Treat trace data as production telemetry with same access controls as logs

In [3]:
# Enable content capture for development
# Both environment variables work - the first is for Azure SDK, second for OpenTelemetry instrumentation
os.environ["AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED"] = "true"
os.environ["OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT"] = "true"

print("✅ Content capture enabled")
print("   Trace data will include full message content (prompts and responses)")
print("\n⚠️  Remember to disable in production if content contains PII")

✅ Content capture enabled
   Trace data will include full message content (prompts and responses)

⚠️  Remember to disable in production if content contains PII


# Part 3: Azure Monitor Tracing (Recommended)

For production AND development, send traces to **Azure Monitor Application Insights**. This enables:
- Viewing traces in the Foundry portal
- Long-term storage and analysis
- Alerting on errors or latency
- Correlation with other Azure services

## Setup Steps

1. Navigate to **Tracing** in the Foundry portal left navigation
2. Create or connect an **Application Insights** resource
3. The SDK retrieves the connection string automatically from your project

In [4]:
# Configure Azure Monitor tracing
# This follows the official Microsoft Foundry documentation

try:
    from azure.monitor.opentelemetry import configure_azure_monitor
    from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor
    AZURE_MONITOR_AVAILABLE = True
except ImportError as e:
    AZURE_MONITOR_AVAILABLE = False
    print(f"⚠️ Missing packages: {e}")
    print("   Install with: uv pip install azure-monitor-opentelemetry opentelemetry-instrumentation-openai-v2")

if AZURE_MONITOR_AVAILABLE:
    # Connect to your Foundry project and get Application Insights connection string
    project_client = AIProjectClient(
        endpoint=endpoint,
        credential=DefaultAzureCredential(),
    )
    
    try:
        # Get Application Insights connection string from project
        connection_string = project_client.telemetry.get_application_insights_connection_string()
        print(f"✅ Application Insights connected")
        print(f"   Connection string: {connection_string[:50]}...")
        
        # Configure Azure Monitor (enables telemetry collection)
        configure_azure_monitor(connection_string=connection_string)
        
        # Instrument the OpenAI SDK - THIS IS KEY!
        # This automatically captures all OpenAI API calls as spans
        OpenAIInstrumentor().instrument()
        
        print("\n✅ Azure Monitor configured")
        print("✅ OpenAI SDK instrumented")
        print("   All OpenAI calls will be automatically traced!")
        print("   View traces in Foundry portal: Project → Tracing")
        
    except Exception as e:
        print(f"⚠️ Could not connect to Application Insights: {e}")
        print("   Make sure Application Insights is connected to your Foundry project")

✅ Application Insights connected
   Connection string: InstrumentationKey=656c9f99-cd7f-48ce-849a-4e8aef4...

✅ Azure Monitor configured
✅ OpenAI SDK instrumented
   All OpenAI calls will be automatically traced!
   View traces in Foundry portal: Project → Tracing


In [5]:
# Test tracing with a simple chat completion
# This demonstrates how traces appear in Application Insights
print("🔍 Testing tracing with chat completion...\n")

from opentelemetry import trace
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

tracer = trace.get_tracer(__name__)

# Show available deployments
print("📋 Available model deployments:")
deployments = list(project_client.deployments.list())
for d in deployments:
    print(f"   - {d.name}: {d.get('sku', {}).get('name', 'N/A')}")

# The AI Services endpoint (without /api/projects/dev)
ai_services_base = endpoint.split("/api/projects")[0]
print(f"\n🔗 AI Services base: {ai_services_base}")
print(f"   Using model deployment: {model}")

# Create an Azure OpenAI client with token credential
token_provider = get_bearer_token_provider(
    DefaultAzureCredential(),
    "https://cognitiveservices.azure.com/.default"
)

openai_client = AzureOpenAI(
    azure_endpoint=ai_services_base,
    api_version="2024-10-21",  # Azure OpenAI API version
    azure_ad_token_provider=token_provider
)

# Make a traced request
with tracer.start_as_current_span("example-tracing") as span:
    span.set_attribute("user.query", "Write a haiku about observability")
    
    response = openai_client.chat.completions.create(
        model=model,  # This is the deployment name
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Keep responses brief."},
            {"role": "user", "content": "Write a haiku about observability."}
        ],
        temperature=0.7,
        max_tokens=100
    )
    
    assistant_response = response.choices[0].message.content
    span.set_attribute("assistant.response", assistant_response[:100] if assistant_response else "")
    
    print(f"✅ Response received!")
    print(f"\n📝 Response:\n{assistant_response}")

print("\n✅ Trace sent to Application Insights!")
print("   View in Foundry portal: Project → Tracing")

🔍 Testing tracing with chat completion...

📋 Available model deployments:
   - gpt-4.1: GlobalStandard
   - text-embedding-3-small: Standard

🔗 AI Services base: https://nw-fy-26.services.ai.azure.com
   Using model deployment: gpt-4.1
✅ Response received!

📝 Response:
Signals whisper truth—  
Logs, traces, metrics converge,  
Clarity unfolds.

✅ Trace sent to Application Insights!
   View in Foundry portal: Project → Tracing


# Part 4: Custom Spans for Business Logic

Now that basic tracing works, let's capture **your own code** with custom spans. This is useful when you have business logic that calls multiple LLM operations.

## Why Custom Spans?

| Without Custom Spans | With Custom Spans |
|---------------------|-------------------|
| See only individual LLM calls | See the full operation that triggered them |
| Hard to correlate related calls | Related calls grouped under parent span |
| No business context | Custom attributes like `claims_count` |

In [13]:
# Get a tracer instance for custom spans
tracer = trace.get_tracer(__name__)

# Example: A function that assesses claims using chat completions
def build_assessment_prompt(claim: str, context: str) -> str:
    """Build a prompt for claim verification."""
    return f"""Evidence: {context}

Assess this claim based on the evidence. Output only 'True', 'False', or 'NEE' (not enough evidence).

Claim: {claim}

Assessment:"""

@tracer.start_as_current_span("assess_claims_with_context")
def assess_claims_with_context(claims: list, contexts: list) -> list:
    """Assess multiple claims against their contexts with full tracing."""
    responses = []
    
    # Add custom attribute to track how many claims we're processing
    current_span = trace.get_current_span()
    current_span.set_attribute("operation.claims_count", len(claims))
    
    # System instructions for claim assessment
    system_message = "You assess claims. Output only 'True', 'False', or 'NEE' (not enough evidence). No explanations."
    
    for i, (claim, context) in enumerate(zip(claims, contexts)):
        # Create a nested span for each assessment
        with tracer.start_as_current_span(f"assess_claim_{i}") as child_span:
            child_span.set_attribute("claim.text", claim[:50])
            
            response = openai_client.chat.completions.create(
                model=model,
                messages=[
                    {"role": "system", "content": system_message},
                    {"role": "user", "content": build_assessment_prompt(claim, context)}
                ],
                temperature=0.0,  # Deterministic for assessment
                max_tokens=10
            )
            
            result = response.choices[0].message.content.strip('., ').strip()
            child_span.set_attribute("assessment.result", result)
            responses.append(result)
    
    return responses

print("✅ Custom traced function defined: assess_claims_with_context")

✅ Custom traced function defined: assess_claims_with_context


In [14]:
# Test the custom traced function
print("🔍 Running claim assessment with tracing...\n")

claims = [
    "The Earth orbits the Sun.",
    "Water boils at 50°C at sea level.",
]
contexts = [
    "The Earth completes one orbit around the Sun every 365.25 days.",
    "At standard atmospheric pressure, water boils at 100°C (212°F).",
]

results = assess_claims_with_context(claims, contexts)

for claim, result in zip(claims, results):
    emoji = "✅" if result == "True" else "❌" if result == "False" else "❓"
    print(f"{emoji} Claim: '{claim[:40]}...' → {result}")

print("\n📎 View the trace hierarchy in Foundry portal")
print("   You'll see: assess_claims_with_context → multiple chat completion spans")

🔍 Running claim assessment with tracing...

✅ Claim: 'The Earth orbits the Sun....' → True
❌ Claim: 'Water boils at 50°C at sea level....' → False

📎 View the trace hierarchy in Foundry portal
   You'll see: assess_claims_with_context → multiple chat completion spans


In [9]:
# Part 5: Flush traces before proceeding
# Ensure all traces are exported to Azure Monitor
print("⏳ Flushing traces to Azure Monitor...")

# The Azure Monitor exporter typically batches and sends traces
# Give it a moment to complete
import time
time.sleep(2)

print("✅ Traces flushed!")
print("   Check the Foundry portal in ~1-2 minutes for trace data")

⏳ Flushing traces to Azure Monitor...
✅ Traces flushed!
   Check the Foundry portal in ~1-2 minutes for trace data


## Alternative: Console Tracing (for CI/CD)

> ℹ️ **Note**: If you don't have Azure Monitor access (e.g., CI/CD pipelines), you can output traces to console instead.

```python
from opentelemetry import trace
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor, ConsoleSpanExporter
from opentelemetry.sdk.resources import Resource

def setup_console_tracing():
    """Configure tracing to output to console (for testing/CI)."""
    resource = Resource.create({"service.name": "my-agent"})
    tracer_provider = TracerProvider(resource=resource)
    tracer_provider.add_span_processor(SimpleSpanProcessor(ConsoleSpanExporter()))
    trace.set_tracer_provider(tracer_provider)
```

Don't run this if you already configured Azure Monitor above.

In [10]:
## Part 5 Continued: Agent-Style Tool Calling with Tracing
# Now let's trace a more complex scenario: an agent that uses tools
# This shows the full trace hierarchy: session → LLM → tool calls

print("🤖 Setting up agent-style tracing with tool calling...")
print("   This demonstrates how to trace complex agent workflows")

🤖 Setting up agent-style tracing with tool calling...
   This demonstrates how to trace complex agent workflows


In [11]:
# Define tool implementations (these would be called when agent uses tools)
def execute_tool(tool_name: str, args: dict) -> str:
    """Execute a tool and return the result."""
    if tool_name == "fetch_weather":
        location = args.get("location", "Unknown")
        weather_data = {
            "Seattle": "14°C, Rainy",
            "New York": "22°C, Sunny",
            "London": "16°C, Cloudy",
        }
        return weather_data.get(location, "20°C, Unknown conditions")
    elif tool_name == "calculate_tip":
        bill = args.get("bill_amount", 0)
        percent = args.get("tip_percent", 18.0)
        tip = bill * (percent / 100)
        return f"Tip: ${tip:.2f}, Total: ${bill + tip:.2f}"
    return "Unknown tool"

print("✅ Tool executor defined: execute_tool(tool_name, args)")

✅ Tool executor defined: execute_tool(tool_name, args)


## Tool Schema Definitions

Define OpenAI function calling schemas so the model knows what tools are available:

In [15]:
# Execute agent-like behavior with tool calling using our traced client
print("🤖 Running agent-style query with tool calling...\n")

# Define tool schemas for OpenAI function calling
tools = [
    {
        "type": "function",
        "function": {
            "name": "fetch_weather",
            "description": "Fetches current weather for a location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {"type": "string", "description": "City name"}
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate_tip",
            "description": "Calculates tip amount for a bill",
            "parameters": {
                "type": "object",
                "properties": {
                    "bill_amount": {"type": "number", "description": "Total bill amount"},
                    "tip_percent": {"type": "number", "description": "Tip percentage", "default": 18.0}
                },
                "required": ["bill_amount"]
            }
        }
    }
]

# Run a traced request with tools available
with tracer.start_as_current_span("agent_session") as span:
    span.set_attribute("user.query", "What's the weather in Seattle?")
    span.set_attribute("agent.tools", ["fetch_weather", "calculate_tip"])
    
    response = openai_client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": "You are a helpful assistant. Use tools when appropriate."},
            {"role": "user", "content": "What's the weather in Seattle and calculate a 20% tip on $85?"}
        ],
        tools=tools,
        tool_choice="auto",
        temperature=0.0
    )
    
    # Log tool calls if any
    if response.choices[0].message.tool_calls:
        tool_names = [tc.function.name for tc in response.choices[0].message.tool_calls]
        span.set_attribute("agent.tool_calls", str(tool_names))
        print(f"📞 Tools called: {tool_names}")
    
    print(f"\n✅ Response received with tool calling enabled")
    print(f"   Check Foundry Tracing for detailed span hierarchy")

print("\n📎 View trace in Foundry portal: Project → Tracing")
print("   You'll see: agent_session → LLM calls with tool definitions")

🤖 Running agent-style query with tool calling...

📞 Tools called: ['fetch_weather', 'calculate_tip']

✅ Response received with tool calling enabled
   Check Foundry Tracing for detailed span hierarchy

📎 View trace in Foundry portal: Project → Tracing
   You'll see: agent_session → LLM calls with tool definitions


# Part 6: View Traces in Microsoft Foundry Portal

## Steps to View Your Traces

1. Navigate to: [ai.azure.com](https://ai.azure.com)
2. Select your project
3. Click **Tracing** in the left navigation
4. Filter by time range (last 15 minutes)
5. Click on any trace to see span hierarchy

## What You'll See

```
┌───────────────────────────────────────────────────────────┐
│  Trace: example-tracing              Duration: ~2.3s      │
├───────────────────────────────────────────────────────────┤
│  ├── chat.completions.create           1500ms            │
│  │   └── attributes: model=gpt-4o                        │
│  │                                                        │
│  └── assess_claims_with_context         900ms            │
│      ├── assess_claim_0                 450ms            │
│      └── assess_claim_1                 450ms            │
└───────────────────────────────────────────────────────────┘
```

> 💡 **Tip**: Allow 1-2 minutes for traces to appear in the portal after running your code.

# Part 7: Local Tracing with AI Toolkit

> ⚠️ **Important**: You cannot use Azure Monitor AND AI Toolkit in the same kernel session. OpenTelemetry only allows one `TracerProvider` per process.

For local development with AI Toolkit, use the dedicated notebook: **`lab9b_aitk_tracing.ipynb`**

## Why a Separate Notebook?

| Issue | Explanation |
|-------|-------------|
| **Single TracerProvider** | OTel SDK limitation - only one exporter target per process |
| **Cannot Switch** | Once `configure_azure_monitor()` runs, traces go to Azure forever |
| **Clean Slate** | The AITK notebook starts fresh, sends to `localhost:4318` |

## Quick Reference: AI Toolkit Setup

1. Install [AI Toolkit for VS Code](https://marketplace.visualstudio.com/items?itemName=ms-windows-ai-studio.windows-ai-studio)
2. Open **Tracing** in AI Toolkit sidebar
3. Click **Start Collector** (must show "Status: Running")
4. Open `notebooks/lab9b_aitk_tracing.ipynb`
5. Run all cells → Click **Refresh** in Tracing view

> 📚 **Documentation**: [Tracing in AI Toolkit](https://code.visualstudio.com/docs/intelligentapps/tracing)

In [ ]:
# This cell is intentionally left as documentation only.
# DO NOT run AI Toolkit setup in this notebook - it will conflict with Azure Monitor.
#
# For AI Toolkit tracing, use: notebooks/lab9b_aitk_tracing.ipynb
#
# The key difference in setup:
#
# AZURE MONITOR (this notebook):
#   from azure.monitor.opentelemetry import configure_azure_monitor
#   configure_azure_monitor(connection_string="InstrumentationKey=...")
#   OpenAIInstrumentor().instrument()
#
# AI TOOLKIT (lab9b_aitk_tracing.ipynb):
#   from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
#   tracer_provider = TracerProvider(resource=Resource.create({"service.name": "my-app"}))
#   tracer_provider.add_span_processor(BatchSpanProcessor(
#       OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
#   ))
#   trace.set_tracer_provider(tracer_provider)
#   OpenAIInstrumentor().instrument()

print("ℹ️  This notebook uses Azure Monitor for tracing.")
print("   For local AI Toolkit tracing, open: lab9b_aitk_tracing.ipynb")

✅ AI Toolkit Tracing Configured!
   Endpoint: http://localhost:4318/v1/traces
   Service: lab9-ai-toolkit-demo

📋 Next steps:
   1. Run the code cells below to generate traces
   2. Click 'Refresh' in AI Toolkit Tracing view
   3. You should see your traces appear!


## Complete Agent Session Example with Tracing

The following example shows a **complete agent loop** with full tracing. Whether you use Azure Monitor or AI Toolkit, you'll see the entire agent workflow:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│  Trace: agent_conversation                                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│  ├── planning (LLM decides what tools to use)                              │
│  │   └── chat.completions.create                                           │
│  │                                                                          │
│  ├── tool_execution (agent calls tools)                                    │
│  │   ├── fetch_weather → "14°C, Rainy"                                     │
│  │   └── calculate_tip → "Tip: $17.00"                                     │
│  │                                                                          │
│  └── response_generation (LLM synthesizes final answer)                    │
│      └── chat.completions.create                                           │
└─────────────────────────────────────────────────────────────────────────────┘
```

This is exactly what you'll see in AI Toolkit's Trace Viewer or Foundry's Tracing portal!

In [21]:
# Complete Agent Session with Full Tracing
# This demonstrates the entire agent loop: plan → execute tools → respond
# Works with both Azure Monitor AND AI Toolkit!

import json
from opentelemetry import trace

tracer = trace.get_tracer(__name__)

def run_traced_agent_session(user_query: str) -> str:
    """
    Run a complete agent session with full tracing.
    
    This shows exactly what happens in an agent:
    1. Planning: LLM decides which tools to call
    2. Tool Execution: Agent runs the tools
    3. Response: LLM generates final answer
    """
    
    # Define our tools (same as before)
    tools = [
        {
            "type": "function",
            "function": {
                "name": "fetch_weather",
                "description": "Get current weather for a city",
                "parameters": {
                    "type": "object",
                    "properties": {"location": {"type": "string"}},
                    "required": ["location"]
                }
            }
        },
        {
            "type": "function", 
            "function": {
                "name": "calculate_tip",
                "description": "Calculate tip for a bill",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "bill_amount": {"type": "number"},
                        "tip_percent": {"type": "number", "default": 18}
                    },
                    "required": ["bill_amount"]
                }
            }
        }
    ]
    
    # Tool implementations
    def execute_tool(name: str, args: dict) -> str:
        with tracer.start_as_current_span(f"tool_{name}") as tool_span:
            tool_span.set_attribute("tool.name", name)
            tool_span.set_attribute("tool.arguments", json.dumps(args))
            
            if name == "fetch_weather":
                result = {"Seattle": "14°C, Rainy", "NYC": "22°C, Sunny"}.get(
                    args.get("location", ""), "20°C, Clear"
                )
            elif name == "calculate_tip":
                bill = args.get("bill_amount", 0)
                pct = args.get("tip_percent", 18)
                tip = bill * (pct / 100)
                result = f"Tip: ${tip:.2f}, Total: ${bill + tip:.2f}"
            else:
                result = "Unknown tool"
            
            tool_span.set_attribute("tool.result", result)
            return result
    
    # === THE AGENT LOOP (fully traced) ===
    with tracer.start_as_current_span("agent_conversation") as session_span:
        session_span.set_attribute("user.query", user_query)
        messages = [
            {"role": "system", "content": "You are a helpful assistant. Use tools when needed."},
            {"role": "user", "content": user_query}
        ]
        
        # STEP 1: Planning - LLM decides what tools to use
        with tracer.start_as_current_span("planning") as plan_span:
            plan_span.set_attribute("step", "planning")
            
            response = openai_client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools,
                tool_choice="auto",
                temperature=0
            )
            
            assistant_msg = response.choices[0].message
            tool_calls = assistant_msg.tool_calls or []
            plan_span.set_attribute("tools_requested", len(tool_calls))
        
        # STEP 2: Tool Execution - Run each tool the agent requested
        if tool_calls:
            with tracer.start_as_current_span("tool_execution") as exec_span:
                exec_span.set_attribute("step", "tool_execution")
                exec_span.set_attribute("tool_count", len(tool_calls))
                
                # Add assistant message with tool calls
                messages.append(assistant_msg)
                
                for tc in tool_calls:
                    tool_name = tc.function.name
                    tool_args = json.loads(tc.function.arguments)
                    
                    print(f"   🔧 Calling tool: {tool_name}({tool_args})")
                    result = execute_tool(tool_name, tool_args)
                    print(f"      → Result: {result}")
                    
                    # Add tool result to conversation
                    messages.append({
                        "role": "tool",
                        "tool_call_id": tc.id,
                        "content": result
                    })
        
        # STEP 3: Response Generation - LLM creates final answer
        with tracer.start_as_current_span("response_generation") as resp_span:
            resp_span.set_attribute("step", "response_generation")
            
            final_response = openai_client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=0.7
            )
            
            final_answer = final_response.choices[0].message.content
            resp_span.set_attribute("response_length", len(final_answer))
        
        session_span.set_attribute("agent.success", True)
        return final_answer


# Run the traced agent session
print("🤖 Running Complete Agent Session with Tracing")
print("=" * 55)
print()

query = "What's the weather in Seattle? Also, calculate a 20% tip on $85."
print(f"📝 User Query: {query}")
print()
print("🔍 Agent Steps:")

result = run_traced_agent_session(query)

print()
print("=" * 55)
print(f"💬 Agent Response:\n{result}")
print()
print("✅ Full trace captured!")
print("   View in: Foundry Portal → Tracing  OR  AI Toolkit → Tracing")
print()
print("📊 Trace Hierarchy:")
print("   agent_conversation")
print("   ├── planning (LLM + tool selection)")  
print("   ├── tool_execution")
print("   │   ├── tool_fetch_weather")
print("   │   └── tool_calculate_tip")
print("   └── response_generation (final LLM call)")

🤖 Running Complete Agent Session with Tracing

📝 User Query: What's the weather in Seattle? Also, calculate a 20% tip on $85.

🔍 Agent Steps:
   🔧 Calling tool: fetch_weather({'location': 'Seattle'})
      → Result: 14°C, Rainy
   🔧 Calling tool: calculate_tip({'bill_amount': 85, 'tip_percent': 20})
      → Result: Tip: $17.00, Total: $102.00

💬 Agent Response:
The weather in Seattle is currently 14°C and rainy.

A 20% tip on $85 is $17.00, making your total $102.00.

✅ Full trace captured!
   View in: Foundry Portal → Tracing  OR  AI Toolkit → Tracing

📊 Trace Hierarchy:
   agent_conversation
   ├── planning (LLM + tool selection)
   ├── tool_execution
   │   ├── tool_fetch_weather
   │   └── tool_calculate_tip
   └── response_generation (final LLM call)


In [22]:
# Enable automatic Agent SDK instrumentation
# This instruments all Azure AI Agents operations automatically

try:
    from azure.ai.agents.telemetry import AIAgentsInstrumentor
    
    # Instrument the Azure AI Agents SDK
    AIAgentsInstrumentor().instrument()
    
    print("✅ AIAgentsInstrumentor enabled")
    print("   All agent operations will be automatically traced")
    print("   Includes: agent creation, message handling, tool calls, runs")
    
except ImportError:
    print("ℹ️  AIAgentsInstrumentor not available (optional)")
    print("   Install with: pip install azure-ai-projects[agents]")
    print("   Manual tracing with spans will still work")

✅ AIAgentsInstrumentor enabled
   All agent operations will be automatically traced
   Includes: agent creation, message handling, tool calls, runs


# Part 8: Advanced Trace Analysis & KQL Queries

## Trace Hierarchy Visualization

You can view detailed span hierarchies in the Foundry portal:

```
┌─────────────────────────────────────────────────────────────────────────────┐
│  Trace: traced_agent_session                           Duration: 2.3s      │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  ├── create_agent                                      150ms               │
│  │   └── attributes: agent.name=AzureMonitorAgent                          │
│  │                                                                          │
│  ├── llm_call (planning)                               800ms               │
│  │   └── attributes: model=gpt-4o, tokens.input=45                         │
│  │                                                                          │
│  ├── tool_call (calculate_tip)                         50ms                │
│  │   └── attributes: tool.args={bill_amount: 85, tip_percent: 20}          │
│  │                                                                          │
│  └── llm_call (response)                               1200ms              │
│      └── attributes: model=gpt-4o, tokens.output=120                       │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

## Conversation Results

A **Conversation** is the persistent context of an end-to-end dialogue history between a user and an agent. In the Foundry portal, you can view Conversation results including:

- Conversation history details
- Response information and tokens in a run
- Ordered actions, run steps, and tool calls
- Inputs and outputs between a user and an agent

Search by Conversation ID, Response ID, or Trace ID to review specific conversations.

## Azure Monitor Query (KQL)

Use Kusto Query Language to analyze traces programmatically:

```sql
-- Find all traces from your service
traces
| where cloud_RoleName == "lab9-azure-monitor-demo"
| order by timestamp desc
| take 100
```

# Part 9: Summary and Key Takeaways

## Core Concepts Recap

### 1. Traces and Spans

```
TRACE (one request end-to-end)
│
├── SPAN: agent_session (root)          ← Your custom span
│   │
│   ├── SPAN: planning                  ← Your custom span
│   │   └── SPAN: chat.completions      ← Auto-instrumented by OpenAIInstrumentor
│   │
│   ├── SPAN: tool_execution            ← Your custom span
│   │   ├── SPAN: tool_weather          ← Your custom span
│   │   └── SPAN: tool_calculator       ← Your custom span
│   │
│   └── SPAN: response_generation       ← Your custom span
│       └── SPAN: chat.completions      ← Auto-instrumented
```

**Key insight**: `OpenAIInstrumentor().instrument()` automatically creates spans for all OpenAI API calls. You add custom spans to provide business context.

### 2. Two Tracing Backends

| Backend | Notebook | Exporter | Use When |
|---------|----------|----------|----------|
| **Azure Monitor** | `lab9_tracing_observability.ipynb` (this) | `configure_azure_monitor()` | Production, team, persistence |
| **AI Toolkit** | `lab9b_aitk_tracing.ipynb` | `OTLPSpanExporter(localhost:4318)` | Local dev, rapid iteration |

⚠️ **Cannot mix**: One `TracerProvider` per process. Choose one per session.

### 3. Auto-Instrumentation vs Custom Spans

| Type | Code | Creates |
|------|------|---------|
| **Auto** | `OpenAIInstrumentor().instrument()` | Spans for every `chat.completions.create()` call |
| **Auto** | `AIAgentsInstrumentor().instrument()` | Spans for Azure AI Agents SDK operations |
| **Custom** | `with tracer.start_as_current_span("name"):` | Your business logic spans |
| **Custom** | `@tracer.start_as_current_span("name")` | Decorator form |

---

## Quick Setup Reference

### Azure Monitor (Production)

```python
# 1. Get connection string from your Foundry project
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.monitor.opentelemetry import configure_azure_monitor
from opentelemetry.instrumentation.openai_v2 import OpenAIInstrumentor

project = AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], 
                          credential=DefaultAzureCredential())
conn_str = project.telemetry.get_application_insights_connection_string()

# 2. Configure Azure Monitor
configure_azure_monitor(connection_string=conn_str)

# 3. Instrument OpenAI SDK (KEY STEP!)
OpenAIInstrumentor().instrument()

# 4. All OpenAI calls now traced automatically!
```

### AI Toolkit (Local Development)

```python
# See lab9b_aitk_tracing.ipynb for complete example
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter

tracer_provider = TracerProvider(resource=Resource.create({"service.name": "my-agent"}))
tracer_provider.add_span_processor(BatchSpanProcessor(
    OTLPSpanExporter(endpoint="http://localhost:4318/v1/traces")
))
trace.set_tracer_provider(tracer_provider)
OpenAIInstrumentor().instrument()
```

---

## Environment Variables

| Variable | Purpose |
|----------|---------|
| `OTEL_INSTRUMENTATION_GENAI_CAPTURE_MESSAGE_CONTENT=true` | Capture prompt/response content |
| `AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED=true` | Same as above (Azure SDK variant) |
| `OTEL_SERVICE_NAME=my-service` | Identify your service in traces |

---

## Framework Integration Summary

| Framework | Instrumentation | Package |
|-----------|----------------|---------|
| **OpenAI SDK** | `OpenAIInstrumentor().instrument()` | `opentelemetry-instrumentation-openai-v2` |
| **Azure AI Agents** | `AIAgentsInstrumentor().instrument()` | `azure-ai-agents` |
| **Semantic Kernel** | Built-in (automatic) | - |
| **LangChain** | `AzureAIOpenTelemetryTracer` callback | `langchain-azure-ai` |
| **Custom Code** | `tracer.start_as_current_span()` | `opentelemetry-sdk` |

---

## Key Takeaways

1. **Traces = Request Journey**: A trace captures everything that happens for one user request
2. **Spans = Individual Operations**: Each LLM call, tool execution, or function is a span
3. **Auto-Instrumentation is Key**: `OpenAIInstrumentor().instrument()` does most of the work
4. **Custom Spans Add Context**: Wrap your business logic to see the full picture
5. **Choose Backend Wisely**: Azure Monitor for prod, AI Toolkit for local dev
6. **One Provider Per Process**: Cannot mix backends in same session

---

## Further Reading

### Microsoft Documentation
- [Trace and observe AI agents](https://learn.microsoft.com/azure/ai-foundry/how-to/develop/trace-observe-agents)
- [Tracing in AI Toolkit](https://code.visualstudio.com/docs/intelligentapps/tracing)

### OpenTelemetry
- [OpenTelemetry Python SDK](https://opentelemetry.io/docs/languages/python/)
- [Semantic Conventions for GenAI](https://opentelemetry.io/docs/specs/semconv/gen-ai/)